In [2]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [3]:
template = 'https://fr.indeed.com/jobs?q={}&l={}'

In [4]:
def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://fr.indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url

In [101]:
url = get_url("data scientist", "ile de france")

### Extract raw HTML

In [102]:
response = requests.get(url)

In [103]:
response

<Response [200]>

In [104]:
response.reason

''

In [105]:
soup = BeautifulSoup(response.text, 'html.parser')

In [106]:
cards = soup.find_all('div', 'job_seen_beacon')

In [168]:
len(cards)

15

### Prototype the model with a single record

In [167]:
card = cards[0]

In [169]:
atag = card.h2.a

In [133]:
atag.get('title')

In [134]:
job_title = atag.get('title') 
#Chercher ou il y a le titre et voir ou l'on voit explicitement 'title' -> dans h2 puis dans la categorie span 

In [135]:
job_url = "https://fr.indeed.com" + atag.get("href")

In [136]:
card.find("span","companyName").text

'OSTENDI'

In [137]:
company = card.find("span","companyName").text
company

'OSTENDI'

In [138]:
job_location = card.find('div', 'companyLocation').text

In [139]:
job_summary = card.find('div','job-snippet').text.strip()

In [140]:
post_date = card.find('span','date').text

In [141]:
today = datetime.today().strftime('%Y-%m-%d')

In [142]:
try:
    job_salary = card.find('div','attribute_snippet').text
except AttributeError:
    job_salary=''

### Generalize the model with a function

In [170]:
def get_record(card):
    """Extract job data from a single record"""
    atag = card.h2.a
    job_url = "https://fr.indeed.com" + atag.get("href")
    company = card.find("span","companyName").text
    job_location = card.find('div', 'companyLocation').text
    job_summary = card.find('div','job-snippet').text.strip()
    post_date = card.find('span','date').text
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('div','attribute_snippet').text
    except AttributeError:
        job_salary=''
    
    record = (company, job_location, post_date, job_summary, job_salary, job_url)
    
    return record

  

In [163]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)

In [164]:
records[0]

(None,
 'KANTAR',
 'Courbevoie (92)',
 "PostedAujourd'hui",
 "Vous êtes diplômé(e) d'une formation en Statistique, Mathématiques ou d’une formation universitaire ou ingénieur en data science.",
 'Temps plein',
 'https://fr.indeed.com/rc/clk?jk=9ff1b39404babdd0&fccid=fc0eeff012676923&vjs=3')

### Getting the next page

In [165]:
while True:
    try:
        url = 'https://fr.indeed.com' + soup.find('a',{'aria-label' : 'Suivant'}).get('href')
    except AttributeError:
        break
        
    response= requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div','job_seen_beacon')
    
    for card in cards:
        record = get_record(card)
        records.append(record)

In [166]:
len(records)

15

### Putting it all together

In [175]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup


def get_record(card):
    """Extract job data from a single record"""
    atag = card.h2.a 
    job_url = "https://fr.indeed.com" + atag.get("href")
    company = card.find("span","companyName").text
    job_location = card.find('div', 'companyLocation').text
    job_summary = card.find('div','job-snippet').text.strip()
    post_date = card.find('span','date').text
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('div','attribute_snippet').text
    except AttributeError:
        job_salary=''
    
    record = (company, job_location, post_date, job_summary, job_salary, job_url)
    
    return record

def main(position, location):
    """Run the main program routine"""
    records = []
    url = get_url(position, location)
    
    
    while True:
        response= requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div','job_seen_beacon')
        
        for card in cards:
            record = get_record(card)
            records.append(record)
    
        try:
            url = 'https://fr.indeed.com' + soup.find('a',{'aria-label' : 'Suivant'}).get('href')
        except AttributeError:
            break
        
    
#We put the url at the end because we started with the url already
#save the job data 

    with open('result.csv', 'w', newline='', encoding='utf-8') as f:
        writer =csv.writer(f)
        writer.writerow(['Company', 'Location', 'PostDate', 'Summary', 'Salary', 'jobUrl'])
        writer.writerows(records)

In [176]:
#run the main program
main('data scientist', 'Paris')